# 실전 문제 해결 (부족한 데이터셋)

In [1]:
! pip install imblearn

In [2]:
import tensorflow as tf
import numpy as np
import random
from imblearn.over_sampling import BorderlineSMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


## 하이퍼 파라미터

In [3]:
EPOCHS = 25

## 모델 정의

In [4]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비 (불균형한 데이터셋)

In [5]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# TODO: 학습 데이터를 Imbalanced small dataset으로 변형하기
x_train_small = list()
y_train_small = list()

for x, y in zip(x_train, y_train):
  if (y==0 and random.randint(0,100)<5) or y ==1 :
    x_train_small.append(x[:])
    y_train_small.append(y)

x_test_small = list()
y_test_small = list()
for x,y in zip(x_test, y_test):
  if y==0 or y==1:
    x_test_small.append(x[:])
    y_test_small.append(y)

x_train = np.stack(x_train_small, axis=0)
y_train = np.stack(y_train_small, axis=0)

x_test = np.stack(x_test_small, axis=0)
y_test = np.stack(y_test_small, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (불균형한 데이터셋)

In [6]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/25
165/165 [==============================] - 5s 26ms/step - loss: 0.3701 - accuracy: 0.9446 - precision: 0.9525 - recall: 0.9912 - val_loss: 1.4552 - val_accuracy: 0.5090 - val_precision: 0.5045 - val_recall: 1.0000
Epoch 2/25
165/165 [==============================] - 4s 24ms/step - loss: 0.1810 - accuracy: 0.9509 - precision: 0.9556 - recall: 0.9946 - val_loss: 1.1609 - val_accuracy: 0.5385 - val_precision: 0.5201 - val_recall: 0.9940
Epoch 3/25
165/165 [==============================] - 4s 24ms/step - loss: 0.1622 - accuracy: 0.9543 - precision: 0.9580 - recall: 0.9956 - val_loss: 0.8206 - val_accuracy: 0.6185 - val_precision: 0.5681 - val_recall: 0.9890
Epoch 4/25
165/165 [==============================] - 4s 23ms/step - loss: 0.1653 - accuracy: 0.9524 - precision: 0.9574 - recall: 0.9942 - val_loss: 0.9782 - val_accuracy: 0.5780 - val_precision: 0.5427 - val_recall: 0.9910
Epoch 5/25
165/165 [==============================] - 4s 24ms/step - loss: 0.1540 - accuracy: 0.9553

## 데이터셋 준비 (BorderlineSMOTE)

In [7]:
# TODO: BorderlineSMOTE 적용하기
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

smote = BorderlineSMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

## Keras API 모델 학습 (BorderlineSMOTE)

In [8]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/25
313/313 [==============================] - 8s 23ms/step - loss: 0.5191 - accuracy: 0.7927 - precision: 0.8379 - recall: 0.7258 - val_loss: 0.5436 - val_accuracy: 0.7650 - val_precision: 0.7176 - val_recall: 0.8740
Epoch 2/25
313/313 [==============================] - 7s 22ms/step - loss: 0.2671 - accuracy: 0.8987 - precision: 0.9354 - recall: 0.8566 - val_loss: 0.5056 - val_accuracy: 0.7780 - val_precision: 0.8294 - val_recall: 0.7000
Epoch 3/25
313/313 [==============================] - 7s 22ms/step - loss: 0.2024 - accuracy: 0.9289 - precision: 0.9579 - recall: 0.8972 - val_loss: 0.4813 - val_accuracy: 0.8130 - val_precision: 0.8021 - val_recall: 0.8310
Epoch 4/25
313/313 [==============================] - 7s 23ms/step - loss: 0.1672 - accuracy: 0.9434 - precision: 0.9671 - recall: 0.9180 - val_loss: 0.7087 - val_accuracy: 0.7820 - val_precision: 0.7256 - val_recall: 0.9070
Epoch 5/25
313/313 [==============================] - 7s 23ms/step - loss: 0.1418 - accuracy: 0.9549